<a href="https://colab.research.google.com/github/cmannnn/california_housing/blob/main/california_housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning Housing Corp. - California Housing

Your task is to predict median house values in Californian districts, given a number of features from these districts.